In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import re
import tensorflow as tf

In [2]:
!pip install transformers

     |████████████████████████████████| 1.8MB 18.5MB/s 
     |████████████████████████████████| 890kB 39.4MB/s 
     |████████████████████████████████| 2.9MB 54.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=3fdd320f54ab6ff74f0ef1649f030f65283194dc2312228f7d9b6048a293a6fe
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


Read hate dataset and convert it into train and test

In [5]:
df = pd.read_csv('filtered_tweets.csv')

In [6]:
df.head()

,Unnamed: 0,tweet,hate_speech_label
0,0,As a woman you shouldn't complain about cle...,0
1,1,boy dats cold...tyga dwn bad for cuffin dat...,0
2,2,You ever fuck a bitch and she start to cry?...,0
3,3,she look like a tranny,0
4,4,The shit you hear about me might be true or...,0


In [7]:
df = df.drop(columns=['Unnamed: 0'])

In [8]:
msk = np.random.rand(len(df)) < 0.8

train = df[msk]
test = df[~msk]

In [9]:
df.groupby('hate_speech_label')['hate_speech_label'].count()

hate_speech_label
0    23228
1     1425
Name: hate_speech_label, dtype: int64

In [10]:
train.groupby('hate_speech_label')['hate_speech_label'].count()

hate_speech_label
0    18626
1     1147
Name: hate_speech_label, dtype: int64

In [11]:
test.groupby('hate_speech_label')['hate_speech_label'].count()

hate_speech_label
0    4602
1     278
Name: hate_speech_label, dtype: int64

In [12]:
train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
test.columns = ['DATA_COLUMN', 'LABEL_COLUMN']


In [13]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [14]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [19]:
# methods to calculate precision recall and f1
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [24]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy'),recall_m, precision_m, f1_m])



In [25]:
hist = model.fit(train_data, epochs=2, validation_data=validation_data)

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Epoch 1/2


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


   1236/Unknown - 1029s 821ms/step - loss: 0.1078 - accuracy: 0.9604 - recall_m: 0.6087 - precision_m: 0.0475 - f1_m: 0.0856

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


1236/1236 [==============================] - 1073s 857ms/step - loss: 0.1077 - accuracy: 0.9604 - recall_m: 0.6087 - precision_m: 0.0475 - f1_m: 0.0856 - val_loss: 0.2212 - val_accuracy: 0.9445 - val_recall_m: 0.7309 - val_precision_m: 0.0520 - val_f1_m: 0.0949
Epoch 2/2
1236/1236 [==============================] - 1058s 856ms/step - loss: 0.0478 - accuracy: 0.9838 - recall_m: 0.7175 - precision_m: 0.0576 - f1_m: 0.1042 - val_loss: 0.2836 - val_accuracy: 0.9369 - val_recall_m: 0.7539 - val_precision_m: 0.0540 - val_f1_m: 0.0985


Read movie subtitle cleansed data and pass for prediction.

In [72]:
df_bk = pd.read_csv('bad_boys.csv')

In [73]:
df_bk.head()

,Unnamed: 0,0
0,0,Mike! What the hell are you doing?
1,1,"It's called driving, Marcus."
2,2,Slow down.
3,3,What? Four minutes?
4,4,Speed up.


In [74]:
pred_sents_bk = df_bk['0'].to_list()

In [75]:
pred_sents_bk[:5]

['Mike! What the hell are you doing?',
 "It's called driving, Marcus.",
 'Slow down.',
 'What? Four minutes?',
 'Speed up.']

In [76]:
last=0
label=0
df_bk_pred = pd.DataFrame(columns=['sentence','label'])
for i in range(32,len(pred_sents_bk),32):
  tf_batch = tokenizer(pred_sents_bk[last:i], max_length=128, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
  labels = ['0','1']
  label = tf.argmax(tf_predictions, axis=1)
  label = label.numpy()
  
  for j in range(32):
    df_bk_pred.loc[len(df_bk_pred.index)] = [pred_sents_bk[last+j],label[j]]

  last=i


In [77]:
df_bk_pred.head()

,sentence,label
0,Mike! What the hell are you doing?,0
1,"It's called driving, Marcus.",0
2,Slow down.,0
3,What? Four minutes?,0
4,Speed up.,0


Save output to a csv

In [78]:
df_bk_pred.to_csv('bert_output_badboys.csv')

In [79]:
df_bk_pred.describe()

,sentence,label
count,1952,1952
unique,1803,2
top,- Yeah.,0
freq,16,1935


In [80]:
df_bk_pred.label.value_counts()

0    1935
1      17
Name: label, dtype: int64

In [81]:
df_bk_pred[df_bk_pred['label']==1]

,sentence,label
314,I wanted him to see the others die.,1
545,But this fool posted a videoof him trying to k...,1
743,"The guy who shot Mike,he's trying to kill me.",1
806,- Knock me out. Hit me.,1
995,"<i>Yes, kill him.</i>",1
1032,I'm gonna shoot you in your face.,1
1095,"- Well, who up there wants to kill you?",1
1379,- I'll kill his ass!,1
1384,Wake your ass up!,1
1389,"- Shit, everybody wanna kill you.",1
